In [12]:
import os
import requests
from PIL import Image
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
import glob
import numpy as np


img_dir           = "/WorkSpace-2/aroy/data/datasets/coco_ooc/OOC_images_selected/"
caption_dir       = "/WorkSpace-2/aroy/data/datasets/coco_ooc/OOC_images_selected_captions/"
captions_filepath = '/WorkSpace-2/aroy/data/OOC/LLM/coco_ooc_captions.npy'

os.makedirs(caption_dir, exist_ok=True)

# hyperparameters
max_new_tokens=20



In [13]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

# processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-6.7b-coco")
# by default `from_pretrained` loads the weights in float32
# we load in float16 instead to save memory
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16) 
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b-coco", torch_dtype=torch.float16) 
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 39.44 GiB total capacity; 21.21 GiB already allocated; 31.94 MiB free; 21.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [3]:


image_path_list = glob.glob(os.path.join(img_dir, '*.jpg'))
image_name_list = ['']*len(image_path_list)
caption_list    = ['']*len(image_path_list)

for indx, image_path in enumerate(image_path_list):
    
    image_name = os.path.splitext(os.path.split(image_path)[1])[0]
    image_name_list[indx] = image_name
    
    image = Image.open(image_path)
    inputs = processor(image, return_tensors="pt").to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()    
    caption_list[indx] = caption
    
captions_dict = {'image_name_list': image_name_list, 'caption_list':caption_list}
np.save(captions_filepath, captions_dict)

In [4]:
# Language model for query

In [5]:

import os
import openai
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = 'opeb_ai_key'

captions_filepath = '/WorkSpace-2/aroy/data/OOC/LLM/coco_ooc_captions.npy'
captions_dict = np.load(captions_filepath, allow_pickle=True)
captions_dict = captions_dict.item()

image_name_list = captions_dict['image_name_list']
caption_list    = captions_dict['caption_list']

In [11]:
for img_name, caption in zip(image_name_list, caption_list):
    print(image_name + ' : ' + caption)

COCO_val2014_000000004296_var1_512_0 : a zebra is standing in a room with a laptop on a table
COCO_val2014_000000004296_var1_512_0 : a tennis players are playing a game on a blue court
COCO_val2014_000000004296_var1_512_0 : a laptop computer sitting on a desk with a giraffe on it
COCO_val2014_000000004296_var1_512_0 : two men riding horses on the beach with a suitcase in the background
COCO_val2014_000000004296_var1_512_0 : a large sheep is standing in a living room with a table and chairs
COCO_val2014_000000004296_var1_512_0 : a baseball game with a slice of pizza on the field
COCO_val2014_000000004296_var1_512_0 : a baseball game with a batter, catcher, umpire and pitcher
COCO_val2014_000000004296_var1_512_0 : a bird sitting on top of a kitchen counter in a remodeled kitchen


In [6]:

query_string = " - is this normal?"
gpt_response_list = ['']*len(caption_list)
for indx, caption in enumerate(caption_list):
    
    content_str = "\"" + caption + "\"" + query_string
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": content_str}
      ]
    )

    response = completion.choices[0].message.content
    gpt_response_list[indx] = response



In [7]:

query_string = " - is this normal? yes or no"
gpt_response_list_YN = ['']*len(caption_list)
for indx, caption in enumerate(caption_list):
    
    content_str = "\"" + caption + "\"" + query_string
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": content_str}
      ]
    )

    response = completion.choices[0].message.content
    gpt_response_list_YN[indx] = response



In [8]:
gpt_response_list_YN

['no',
 'Yes, it is normal for tennis players to play a game on a court, although not all courts are blue.',
 'no',
 'No',
 'no',
 'no',
 'Yes',
 'no']

In [9]:
query_string = " - is this normal? percentage of possibility"
gpt_response_list = ['']*len(caption_list)
for indx, caption in enumerate(caption_list):
    
    content_str = "\"" + caption + "\"" + query_string
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": content_str}
      ]
    )

    response = completion.choices[0].message.content
    gpt_response_list[indx] = response
    
gpt_response_list

['It is not normal for a zebra to be standing in a room with a laptop on a table. The percentage of possibility is extremely low, probably less than 1%. Zebras are wild animals typically found in grasslands and savannas, not in human environments like rooms with technology.\n',
 "Yes, it is normal for tennis players to play a game on a blue court. Blue courts are quite common, particularly in hard court surfaces, such as those used in the Australian Open and the US Open. It's difficult to assign an exact percentage of possibility, but it is certainly a common occurrence in the world of tennis.",
 "It is not normal to see a laptop computer sitting on a desk with a giraffe on it. This could be interpreted as a small toy or figurine of a giraffe, which would be more plausible. However, if it's meant to be a real-life giraffe, that would be highly improbable. \n\nIn the case of a small toy or figurine: 75%\nIn the case of a real-life giraffe: <1%",
 "It is not common to see two men riding 

In [10]:
query_string = " - how likely is this in a scale between 0 to 10"

gpt_response_list = ['']*len(caption_list)
for indx, caption in enumerate(caption_list):
    
    content_str = "\"" + caption + "\"" + query_string
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": content_str}
      ]
    )

    response = completion.choices[0].message.content
    gpt_response_list[indx] = response
    
gpt_response_list

['I would say this is a 1 or 2 on a scale of 0 to 10, as it is highly unlikely but still within the realm of possibility if it were set up intentionally for entertainment or an art installation.',
 '7',
 'I would say it is a 2. While it is possible to have a giraffe figurine or a giraffe sticker on a laptop, having an actual giraffe on a laptop is highly unlikely due to size and weight constraints.',
 "I would say a 3, as it's not impossible but rather uncommon to see men riding horses on the beach with a suitcase in the background.",
 '3',
 'I would say a 2, as it is quite unlikely but not impossible that someone might accidentally drop a slice of pizza on the field.',
 '10',
 "It is difficult to provide a precise number on the likelihood of this, as it depends on a number of factors such as whether the bird is a pet or managed to fly in accidentally, or if the kitchen is open to the outdoors in some way. I would say it's a 4 or 5 out of 10, as it's not completely unlikely but also no

In [ ]:
query_string = " - whats is the most likely object that is out of context?"

gpt_response_list = ['']*len(caption_list)
for indx, caption in enumerate(caption_list):
    
    content_str = "\"" + caption + "\"" + query_string
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": content_str}
      ]
    )

    response = completion.choices[0].message.content
    gpt_response_list[indx] = response
    
gpt_response_list